# 多输入和多输出通道

In [1]:
import torch
from torch import nn

In [2]:
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    rs = []
    for x, k in zip(X, K):
        print(x.shape, k.shape)
        r = d2l.corr2d(x, k)
        print(r.shape)
        rs.append(r)
    return sum(rs)

In [3]:
# zip 会把多个可迭代对象（如列表、元组、张量等）中对应位置的元素打包成一个个元组，并返回由这些元组组成的迭代器。
a = torch.tensor([[1,2]])
b = torch.tensor([[3,4]])
zipped = list(zip(a, b))
print(zipped)  # 输出: [(tensor([1, 2]), tensor([3, 4]))]

[(tensor([1, 2]), tensor([3, 4]))]


In [4]:
X = torch.tensor([
    [
        [0.0, 1.0, 2.0],
        [3.0, 4.0, 5.0],
        [6.0, 7.0, 8.0]
    ],
    [
        [1.0, 2.0, 3.0],
        [4.0, 5.0, 6.0],
        [7.0, 8.0, 9.0]
    ]
])
K = torch.tensor([
    [
        [0.0, 1.0],
        [2.0, 3.0]
    ],
    [
        [1.0, 2.0],
        [3.0, 4.0]
    ]
])
print(X.shape,K.shape)
corr2d_multi_in(X, K).shape

torch.Size([2, 3, 3]) torch.Size([2, 2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])


torch.Size([2, 2])

In [5]:
def mutil_corr2d_in_out(X, K):
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

In [6]:
torch.stack([torch.tensor([[1,2]]), torch.tensor([[2,3]])],1)

# 这样写是不行的，因为 (3,4) 是一个元组，直接 for x, k in (3, 4) 会导致拆包错误。
# 正确做法，应该传入一组可拆包为 (x, k) 的子元组，例如：
for x, k in [(3, 4), (5, 6)]:
    print(x, k)

3 4
5 6


In [7]:
K = torch.stack((K, K+1, K+2),0)
print(K.shape)
print('---')
mutil_corr2d_in_out(X,K)

torch.Size([3, 2, 2, 2])
---
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])


tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [8]:
mutil_corr2d_in_out(X,K).shape

torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([3, 3]) torch.Size([2, 2])
torch.Size([2, 2])


torch.Size([3, 2, 2])

In [9]:
def corr2d_mutil_in_out_1x1(X,K):
    c_i, h,w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K,X)
    return Y.reshape((c_o, h,w))

In [10]:
X = torch.normal(0,1,(3,3,3))
K=torch.normal(0,1,(2,3,1,1))

Y1 = corr2d_mutil_in_out_1x1(X,K)
Y2 = mutil_corr2d_in_out(X,K)
Y1,Y2
assert torch.abs(Y1-Y2).sum() <1e-6

torch.Size([3, 3]) torch.Size([1, 1])
torch.Size([3, 3])
torch.Size([3, 3]) torch.Size([1, 1])
torch.Size([3, 3])
torch.Size([3, 3]) torch.Size([1, 1])
torch.Size([3, 3])
torch.Size([3, 3]) torch.Size([1, 1])
torch.Size([3, 3])
torch.Size([3, 3]) torch.Size([1, 1])
torch.Size([3, 3])
torch.Size([3, 3]) torch.Size([1, 1])
torch.Size([3, 3])


In [ ]:
import torch
import torch.nn.functional as F

# 示例：3x3 卷积 + 3x3 卷积
k1 = torch.randn(1, 1, 3, 3)  # 第一个卷积核
k2 = torch.randn(1, 1, 3, 3)  # 第二个卷积核
X = torch.randn(1, 1, 10, 10) # 输入

# 方法1：两次卷积
y = F.conv2d(X, k1, padding=0)
z1 = F.conv2d(y, k2, padding=0)

# 方法2：计算等效卷积核
# 等效核 = k2 和 k1 的卷积
# 计算等效卷积核时，将k2在高和宽维度flip一下，是因为conv2d本身实现的是互相关操作（cross-correlation），而不是数学上的卷积（convolution），卷积与互相关差了一个核的180度翻转（即先在空间上flip）。所以，这里k2.flip([2, 3])就是将k2在最后两个空间维度（高和宽）都翻转，实现真正的卷积操作，获得等效核。
k_eq = F.conv2d(k1, k2.flip([2, 3]), padding=2)  # 需要flip做互相关
z2 = F.conv2d(X, k_eq, padding=0)

print("两次卷积输出形状:", z1.shape)
print("单次卷积输出形状:", z2.shape)
print("结果是否接近:", torch.allclose(z1, z2, atol=1e-5))
print(k_eq.shape)

两次卷积输出形状: torch.Size([1, 1, 6, 6])
单次卷积输出形状: torch.Size([1, 1, 6, 6])
结果是否接近: True
torch.Size([1, 1, 5, 5])


In [15]:
torch.tensor([
    [
        [
            [1,2]
        ],
        [
            [3,4]
        ]
    ]
]).flip([2,3])

tensor([[[[2, 1]],

         [[4, 3]]]])

In [16]:
import torch

# 1D 例子
a = torch.tensor([1, 2, 3, 4, 5])
print("原始:", a)
print("翻转:", a.flip([0]))  # [5, 4, 3, 2, 1]

# 2D 例子
b = torch.tensor([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])
print("\n原始:\n", b)
print("\n上下翻转 flip([0]):\n", b.flip([0]))
print("\n左右翻转 flip([1]):\n", b.flip([1]))
print("\n上下+左右翻转 flip([0,1]):\n", b.flip([0, 1]))

原始: tensor([1, 2, 3, 4, 5])
翻转: tensor([5, 4, 3, 2, 1])

原始:
 tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

上下翻转 flip([0]):
 tensor([[7, 8, 9],
        [4, 5, 6],
        [1, 2, 3]])

左右翻转 flip([1]):
 tensor([[3, 2, 1],
        [6, 5, 4],
        [9, 8, 7]])

上下+左右翻转 flip([0,1]):
 tensor([[9, 8, 7],
        [6, 5, 4],
        [3, 2, 1]])


In [21]:
# view 和 reshape 的区别演示
a = torch.rand((3, 4))
b_view = a.view(2, 6)       # view 要求原内存连续
b_reshape = a.reshape(2, 6) # reshape 更灵活，可以自动处理非连续内存情况

print("原始 a 形状:", a.shape)
print("view 后形状:", b_view.shape)
print("reshape 后形状:", b_reshape.shape)

原始 a 形状: torch.Size([3, 4])
view 后形状: torch.Size([2, 6])
reshape 后形状: torch.Size([2, 6])


In [24]:
x = torch.randn(3, 4, 5, 3)
x = x.view(3, -1) 
x.shape

torch.Size([3, 60])